In [1]:
import re, math
import operator
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import StratifiedShuffleSplit
from random import randint
import ipynb.fs.defs.PeopleInfo as peopleInfo
from nltk.corpus import stopwords
from nltk import TweetTokenizer
from nltk import PorterStemmer
# import ipynb.fs.defs.TweetTextHandler as tweetTextHandler
import ipynb.fs.defs.FilterMethods as filterMethods
import sys, os
sys.path.append('../2_feature')
import ipynb.fs.defs.GetFeatures as getFeatures
global stop_words
stop_words = stopwords.words('english')
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from IPython.display import display, Image
plotly.tools.set_credentials_file(username='Adeline', api_key='Z5eltNtBQXqvI05ZFQtz')
# import plotly.offline as offline
# offline.init_notebook_mode(connected=True)

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Text Preprocessing

In [2]:
def replace_by_symbols(txt):
    txt = re.sub(r"https\S+", '', txt)
    txt = re.sub(r"http\S+", '', txt)
    txt = re.sub(r"pic.twitter.com\S+", '', txt)
    txt = re.sub(r"twitter.com/\S+", '', txt)
    txt = re.sub(r"\S+/\S+", '', txt)
    txt = re.sub(r"@\S+", '', txt)
    txt = re.sub(r"#\S+", '', txt)
    txt = re.sub(r"idk", 'i do not know', txt)   # idk: i don't know
    txt = re.sub(r"tho", 'though', txt)   # tho
    txt = re.sub(r"i\'m", 'i am', txt)
    txt = re.sub(r"you\'re", 'you are', txt)
    txt = re.sub(r"he\'s", 'he is', txt)
    txt = re.sub(r"she\'s", 'she is', txt)
    txt = re.sub(r"it\'s", 'it is', txt)
    txt = re.sub(r"we\'re", 'we are', txt)
    txt = re.sub(r"they\'re", 'they are', txt)
    txt = re.sub(r"isn\'t", 'is not', txt)
    txt = re.sub(r"don\'t", 'do not', txt)
    txt = re.sub(r"doesn\'t", 'does not', txt)
    txt = re.sub(r"didn\'t", 'did not', txt)
    txt = re.sub(r"wasn\'t", 'was not', txt)
    txt = re.sub(r"weren\'t", 'were not', txt)
    txt = re.sub(r"haven\'t", 'have not', txt)
    txt = re.sub(r"can\'t", 'can not', txt)
    txt = re.sub(r"couldn\'t", 'could not', txt)
    txt = re.sub(r"wouldn\'t", 'would not', txt)
    txt = re.sub(r"shouldn\'t", 'should not', txt)
    txt = re.sub(r"&amp", '', txt)
    return txt

In [3]:
def tokenize(text):
    text_list = re.findall('(?u)\\b[a-zA-Z]\\w{0,}\\b', text)
    return text_list

# Create LIWC Dictionary

In [6]:
liwc_category_dict = dict()
liwc_word_dict = dict()
with open('/home/adeline/Documents/Depression_Research/LIWC2007 Documents/Dictionaries/LIWC2007_English080730.dic') as open_file:
    raw_data = open_file.readlines()
    for row in raw_data[1:65]:
        row = row.strip().split('\t')
        liwc_category_dict[row[0]] = row[1]
    for row in raw_data[66:4553]:
        row = row.strip().split('\t')
        liwc_word_dict[row[0]] = row[1:]

In [7]:
for stop_word in stop_words:
    try:
        print('{0}\t{1}'.format(stop_word, liwc_word_dict[stop_word]))
    except KeyError:
        print('KeyError: {0}'.format(stop_word))

i	['1', '2', '3', '4']
me	['1', '2', '3', '4']
my	['1', '2', '3', '4']
myself	['1', '2', '3', '4']
we	['1', '2', '3', '5', '121', '131', '138']
our	['1', '2', '3', '5', '121']
ours	['1', '2', '3', '5', '121']
ourselves	['1', '2', '3', '5', '121']
you	['1', '2', '3', '6', '121']
your	['1', '2', '3', '6', '121']
yours	['1', '2', '3', '6', '121']
KeyError: yourself
KeyError: yourselves
he	['1', '2', '3', '7', '121']
him	['1', '2', '3', '7', '121']
his	['1', '2', '3', '7', '121']
himself	['1', '2', '3', '7', '121']
she	['1', '2', '3', '7', '121']
her	['1', '2', '3', '7', '121']
hers	['1', '2', '3', '7', '121']
herself	['1', '2', '3', '7', '121']
it	['1', '2', '9']
its	['1', '2', '9']
itself	['1', '2', '9']
they	['1', '2', '3', '8', '121']
them	['1', '2', '3', '8', '121']
KeyError: their
KeyError: theirs
themselves	['1', '2', '3', '8', '121']
what	['1', '2', '9']
which	['1', '2', '9']
who	['1', '2', '9', '121']
whom	['1', '2', '9', '121']
this	['1', '2', '9']
that	['1', '2', '9']
these	['1'

# Prepare Data 

In [4]:
patients = dict()
ordinarys = dict()
with open('../0_dataset/patient_ids') as r:
    for patient in r.readlines()[:100]:
        patient = patient.strip()
        patients[patient] = peopleInfo.Patient(patient)
with open('../0_dataset/ordinary_ids') as r:
    for ordinary in r.readlines()[:100]:
        ordinary = ordinary.strip()
        ordinarys[ordinary] = peopleInfo.Ordinary(ordinary)

In [6]:
patients = filterMethods.filter_user_by_tweet_number(patients)
ordinarys = filterMethods.filter_user_by_tweet_number(ordinarys)

Remove users:[]
Remove users:[]


In [7]:
base_texts = []
group_texts = []

for key in patients.keys():
    group_texts.append(replace_by_symbols('\n'.join(patients[key].getText())))

for key in ordinarys.keys():
    base_texts.append(replace_by_symbols('\n'.join(ordinarys[key].getText())))

corpus = base_texts + group_texts

# Simple Data Statistic

In [43]:
def df_filter(df):
    # 過濾空列
    filter = df['Text'] != ''
    df = df[filter]
    return df

In [44]:
base_tweets = []    # one element presented one tweet
group_tweets = []
for line in group_texts:
    group_tweets.extend(line.split('\n'))
for line in base_texts:
    base_tweets.extend(line.split('\n'))

In [60]:
# tweet_group_len = [len(tokenize(x)) for x in group_tweets]
# tweet_base_len = [len(tokenize(x)) for x in base_tweets]
tweet_group_len = [len(x.split()) for x in group_tweets]
tweet_base_len = [len(x.split()) for x in base_tweets]

In [61]:
dfGroupTweets = df_filter(pd.DataFrame({'Text':group_tweets,'len':tweet_group_len}))
dfBaseTweets = df_filter(pd.DataFrame({'Text':base_tweets,'len':tweet_base_len}))

In [69]:
print('=== Brief Info of Group Tweets ===')
dfGroupTweets['len'].describe()

=== Brief Info of Group Tweets ===


count    99668.000000
mean        11.514328
std          8.246411
min          0.000000
25%          4.000000
50%         10.000000
75%         18.000000
max         42.000000
Name: len, dtype: float64

In [70]:
print('=== Brief Info of Base Tweets ===')
dfBaseTweets['len'].describe()

=== Brief Info of Base Tweets ===


count    11908.000000
mean        10.626386
std          6.390020
min          0.000000
25%          5.000000
50%          9.000000
75%         15.000000
max         33.000000
Name: len, dtype: float64

# Analyse Sentence

In [8]:
def construct_stopwrods_dict(text_list, mydict):

    position = []
    for i, text in enumerate(text_list):
        if text in stop_words:    # 是個 stopword
            position.append(i)
        else:
            continue
    
    for i in range(len(position)):
        try:
            cur_pos = position[i]
            next_pos = position[i+1]
            key = '{0}-{1}-{2}'.format(text_list[cur_pos], text_list[next_pos], str(next_pos-cur_pos))
            mydict[key] = mydict.get(key, 0) + 1
        except IndexError:
            continue
    
    
    return mydict

In [9]:
mydict = dict()
for texts in group_texts:
    for text in texts.split('\n'):
        construct_stopwrods_dict(tokenize(text), mydict)

In [10]:
mydict2 = dict()
for texts in base_texts:
    for text in texts.split('\n'):
        construct_stopwrods_dict(tokenize(text), mydict2)

In [15]:
%store mydict >> stopwords_dict_depression

Writing 'mydict' (dict) to file 'stopwords_dict_depression'.


In [16]:
%store mydict2 >> stopwords_dict_ordinary

Writing 'mydict2' (dict) to file 'stopwords_dict_ordinary'.
